In [58]:
import numpy as np
import time

def decSVD( A, dindx = None, U = None, s = None, Vt = None, a = None ):

    try:
        if A is None:
            return None, None, None, None, None
    except NameError:
        print ( "No matrix defined!")
        return None, None, None, None, None

    
    m, n = A.shape
    k = min( m, n )

    # calculate the \bar{a} - mean of the original matrix A
    ab = np.dot( A, np.ones( (n, 1 ), dtype=complex ) ) / m
    
    
    # if no SVD decomposition (and mean) is provided, run a full SVD
    if U is None or s is None or Vt is None or a is None:
        
        # ...and the mean centered dataset \bar{A}
        Ab = A - np.dot( ab, np.ones( (1,n) ) );

        Un, sn, Vtn = np.linalg.svd( Ab, full_matrices=False )
        
        return A, Un, sn, Vtn, ab
    
    #if no index for entries to delete is provided, return the inputs 
    if dindx is None or indx.size == 0:
        return A, U, s, Vt, a
    else:
        Ad = np.delete( A, dindx, 1)
        US = np.dot( U, S )
        
        # ...number of remaining columns
        l = Ad.shape[ 1 ]
        
        

        # \widetilde{V}^T matrix with corresponding entries droped (columns below as .svd() provides V transposed, i.e. Vt)
        Vttld = np.delete( Vt, indx, 1 )

        # calculate the \bar{v}^T mean and center the Vttld matrix
        vtb = np.dot( Vttld, np.ones( (l, 1), dtype=complex ) ) / l

        # calculate \hat{V}^T = \widetilde{V}^T - \bar{v}^T x 1
        Vthat = Vttld - np.dot( vtb, np.ones( (1, l), dtype=complex ) )
                        
        # we need the QR-decomposition of the \hat{V} matrix (given \hat{V}^T, we obtain R^T, Q^T)
        Rt, Qt = np.linalg.qr( Vthat )

        # SVD of U \Sigma R^T
        Ud, sd, Vtt = np.linalg.svd( np.dot( US, Rt ), full_matrices=False )
        # ...and last the (V^d)^T component of the SVD
        Vdt = np.dot( Vtt, Qt )

        # ...\Delta{a}^d = U \Sigma \bar{v}
        Da = np.dot( US, vtb )
        # ...and finally \bar{a}^d = \bar{a} + \Delta{a}^d
        ad = ab+Da

        return Ad, Ud, sd, Vdt, ad

In [59]:
m = np.random.randint( 5, 1000 )
n = np.random.randint( 5, 1000 )

k = min( m, n )

# generate a random matrix of size m, n
A = np.random.randn( m, n ) + 1j*np.random.randn( m, n )


# calculate a first SVD decomposition of A
A, U, s, Vt, a = decSVD( A )
S = np.zeros(( k, k ), dtype=complex)
S[:k, :k] = np.diag(s)



indx = np.random.choice( range(n), np.random.randint(1,n/2), replace=False )

t0 = time.time()
Ad = np.delete( A, indx, 1)
And, Uf, sf, Vtf, af = decSVD( Ad )
t1 = time.time()

td0 = time.time()
Ad, Ud, sd, Vdt, ad = decSVD( A, indx, U, s, Vt, a )
td1 = time.time()



l = Ad.shape[ 1 ]

# ...updated \Sigma^d component of the SVD
kn = min( m, l )
Sd = np.zeros(( kn, kn ), dtype=complex)
Sd[:kn, :kn] = np.diag(sd)

AdSVD = np.dot( Ud, np.dot( Sd, Vdt ) ) + np.dot( ad, np.ones( (1,l)))


print( m, n, indx.size, np.allclose( Ad, AdSVD), " SVD: ", t1-t0, " Dec SVD: ", td1-td0 )


735 913 74 True  SVD:  0.34996724128723145  Dec SVD:  0.8190429210662842
